In [ ]:
# data 불러오기
% cd /content
data = []
with open("data.txt") as f:
  for line in f.readlines():
    data.append(line.strip())

len(data)

/content


11700

In [ ]:
!pip install tensorflow==1.15

In [ ]:
import tensorflow as tf
print(tf.__version__)
# 2.x 이 출력될 경우, 런타임 재시작 후 다시 확인

1.15.0


In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
# 드라이브 내 test 파일 경로 설정
% cd /content/drive/MyDrive/Colab Notebooks

# 현재 디렉토리 내 파일 출력
% ls 

In [ ]:
from tokenizationK import FullTokenizer
# ETRI 버트를 받으면 tokenization.py가 들어있는데 그거 말고 꼭 제가 제공해드린 tokenizationK.py를 쓰도록 해주세요.
tokenizer = FullTokenizer(vocab_file="/content/drive/MyDrive/Colab Notebooks/vocab.korean.rawtext.list")

In [ ]:
import re

class Seq:
  def __init__(self, data):
    self.data = self.prep(data)
    self.seq_in = []
    self.index = 0
    self.is_slot = ""
    self.seq_out = []
    self.unmatched_word = ""
    self.has_space = False

  def prep(self, data):
    # "/",";" 이외의 특수문자 제거
    pat = re.compile("[^\w\s/;]")
    data = pat.sub("",data)
    # 슬롯 뒤에 띄어쓰기 추가
    pat = re.compile("/")
    data = pat.sub("/ ",data)
    # 이중공백을 공백으로 변환
    pat = re.compile("\s{2,}")
    data = pat.sub(" ",data)
    return data.lower()

  def get_seq_in(self):
    data = self.data

    # 태그 제거
    pat = re.compile("/\s\w+;")
    data = pat.sub("",data)
    # 특수문자 제거
    pat = re.compile("[^\w\s]")
    data = pat.sub("",data)
    
    data = tokenizer.tokenize(data)
    self.seq_in = data

    return data


  def get_seq_out(self):
    self.data = re.sub(" ","_",self.data)
    for token in self.seq_in:
      if self.index >= len(self.data):
        break
      self.seq_out.append(self.get_slot(token))
      ####print(self.seq_out)
    return self.seq_out

  def slot_end(self):
    subsentence = self.data[self.index:]
    end_m = re.match("/",subsentence)
    if end_m:
      self.index+=1
      self.is_slot = ""
      if self.has_space:
        self.index+=1
        self.has_space = False


  def find_match(self,token):
    matched_word = ""
    pat = re.compile(token)
    subsentence = self.data[self.index:]
    m = pat.match(subsentence)
    if not m:
      if token[-1] == "_":
        matched_word = token[:-1]
        self.has_space = True
      else:
        if self.unmatched_word:
          matched_word = combine(self.unmatched_word,token)
          self.unmatched_word = ""
        else:
          self.unmatched_word = token
    else:
      matched_word = m.group()
    return matched_word


  def slot_start(self):
    subsentence = self.data[self.index:]
    if subsentence[0]=='/':
      temp = re.match("/\w+;",subsentence).group()
      self.index+=len(temp)
      self.is_slot = temp[1:-1]

  def get_slot(self, token):
    pat = re.compile(token)
    if self.is_slot:
      self.slot_end()
    if not self.is_slot:
      self.slot_start()
    matched_word = self.find_match(token)
    self.index+=len(matched_word)

    if self.is_slot:
      return self.is_slot[1:]
    return 0

In [ ]:
# 종성 리스트. 00 ~ 27 + 1(1개 없음)
JONGSUNG_LIST = [' ', 'ㄱ', 'ㄲ', 'ㄳ', 'ㄴ', 'ㄵ', 'ㄶ', 'ㄷ', 'ㄹ', 'ㄺ', 'ㄻ', 'ㄼ', 'ㄽ', 'ㄾ', 'ㄿ', 'ㅀ', 'ㅁ', 'ㅂ', 'ㅄ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']

def combine(word, jongsung):
  if jongsung == 'ᆫ':
    jongsung = 'ㄴ'
  if jongsung == 'ᆼ':
    jongsung = 'ㅇ'
  if jongsung == '[UNK]':
    jongsung = 'ㅂ'
  if jongsung == 'ᆷ':
    jongsung = 'ㅁ'
  if jongsung == 'ᆯ':
    jongsung = 'ㄹ'
  return word[:-1]+chr(ord(word[-1])+JONGSUNG_LIST.index(jongsung))

In [ ]:
seq_in = []
seq_out = []

for sentence in data:
  s = Seq(sentence)
  seq_in.append(s.get_seq_in())
 # try:
  seq_out.append(s.get_seq_out())
  #except:
  #  print(s.seq_in)

In [ ]:
import random

index = list(range(len(seq_in)))

i = random.choice(index)

for j in range(len(seq_in[i])):
  print(seq_in[i][j], seq_out[i][j])

In [ ]:
import pandas as pd

seq = pd.DataFrame({"seq_in":seq_in, "seq_out":seq_out})
seq.to_pickle("seq.pkl")